# Welcome to the Santa 2023 - The Polytope Permutation Puzzle! #

Santa's tree is adorned with many-colored, puzzle-like decorations of all shapes and sizes. Unfortunately, the colors on these decorations have gotten all mixed up! In this competition, you're challenged to set the decorations right in the fewest number of moves.

This notebook will give you a quick introduction to the competition.

# Setup #

In [6]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

# kaggle data_dir
#data_dir = Path("/kaggle/input/santa-2023")

# local data_dir
data_dir = Path("data")

puzzle_info = pd.read_csv(data_dir / 'puzzle_info.csv', index_col='puzzle_type')
# Parse allowed_moves
puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(literal_eval)

puzzles = pd.read_csv(data_dir / 'puzzles.csv', index_col='id')
# Parse color states
puzzles = puzzles.assign(
    initial_state=lambda df: df['initial_state'].str.split(';'),
    solution_state=lambda df: df['solution_state'].str.split(';')
)

sample_submission = pd.read_csv(data_dir / 'sample_submission.csv', index_col='id')

# Permutation Puzzles #

A **permutation puzzle** is an arrangement of **colors** together with a set of permutations of these arrangements called **moves**. The puzzle starts out with an **initial state** of colors and must be permuted through a sequence of moves to its **solution state**. The goal is to do this with as few moves as possible.

In [7]:
# Here is a very simple puzzle
solution_state = ['R', 'G', 'B']
initial_state = ['B', 'G', 'R']
moves = {
    'r': [1, 2, 0],
    's': [1, 0, 2],
}
r = moves['r']
s = moves['s']

The moves are given in "array form", where a move `m` will send position `i` to `m[i]`. You can apply a move in array form using NumPy indexing.

In [8]:
# First convert the state to a NumPy array to use NumPy indexing
initial_state_np = np.asarray(initial_state)

initial_state_np[r]

array(['G', 'R', 'B'], dtype='<U1')

You can also represent a move as a permutation in "disjoint cycle" form.

In [9]:
rp = Permutation(r)
sp = Permutation(s)
print('r:', rp, "\tSends 0 -> 1, 1 -> 2, and 2 -> 0.")
print('s:', sp, "\tSends 0 -> 1, 1 -> 0, and 2 stays fixed." )

# A Permutation is a function which you can apply to a state with a function call
print("Effect of r:", rp(initial_state))
print("Effect of s:", sp(initial_state))

r: (0 1 2) 	Sends 0 -> 1, 1 -> 2, and 2 -> 0.
s: (2)(0 1) 	Sends 0 -> 1, 1 -> 0, and 2 stays fixed.
Effect of r: ['G', 'R', 'B']
Effect of s: ['G', 'B', 'R']


You are also allowed to use the *inverse* of any of a puzzle's moves. The inverse of a permutation just applies the change with "the arrows reversed".

In [10]:
# Use np.argsort to get the inverse using array form
r_inv = np.argsort(r).tolist()
s_inv = np.argsort(s).tolist()
print(f"{r_inv=}, {s_inv=}\n")

# Use a negative power to get the inverse of a Permutation
rp_inv = rp ** -1
sp_inv = sp ** -1

# It's the same permutation either way
assert Permutation(r_inv) == rp_inv
assert Permutation(s_inv) == sp_inv

# In this case, s is equal to its inverse
assert (s == s_inv) and (sp == sp_inv)
# But r is not
assert (r != r_inv) and (rp != rp_inv)

# Inversion reverses the arrows
print('r:', rp_inv, "\tSends 1 -> 0, 2 -> 1, and 0 -> 2.")
print('s:', sp, "\tSends 1 -> 0, 0 -> 1, and 2 stays fixed." )

r_inv=[2, 0, 1], s_inv=[1, 0, 2]

r: (0 2 1) 	Sends 1 -> 0, 2 -> 1, and 0 -> 2.
s: (2)(0 1) 	Sends 1 -> 0, 0 -> 1, and 2 stays fixed.


Here's the solution to this simple puzzle:

In [11]:
# Using array form
state = np.asarray(initial_state)
state = state[s]
state = state[r_inv]
state = state.tolist()
assert state == solution_state

# Using Permutations
state = sp(initial_state)
state = rp_inv(state)
assert state == solution_state

# Cube Puzzles #

There are three puzzle types: `cube`, `wreath`, and `globe`. Each type of puzzle represents its arrangements on some geometric figure with the permutations being a twist or turn of some portion of the figure.
Here, for example, is a `cube_2/2/2` puzzle: a cube with each face sliced into four "facelets". We can see the positions of the facelets as well as a coloring representing a "solved" puzzle.

```
         Positions                                Solution State

         +--------+                               +--------+
         | 0    1 |                               | A    A |
         |   d1   |                               |   d1   |
         | 2    3 |                               | A    A |
+--------+--------+--------+--------+    +--------+--------+--------+--------+
| 16  17 | 4    5 | 8   9  | 12  13 |    | E    E | B    B | C    C | D    D |
|   r1   |   f0   |   r0   |   f1   |    |   r1   |   f0   |   r0   |   f1   |
| 18  19 | 6    7 | 10  11 | 14  15 |    | E    E | B    B | C    C | D    D |
+--------+--------+--------+--------+    +--------+--------+--------+--------+
         | 20  21 |                               | F    F |
         |   d0   |                               |   d0   |
         | 22  23 |                               | F    F |
         +--------+                               +--------+
```

The `2/2/2` means the cube has two layers along each of the three face axes: `d0, d1` (up and down), `f0, f1` (front and back), `r0, r1` (right and left). A `3/3/3` cube will have three layers along each exis, etc. In general, opposing faces will be the layers `a0, a{n-1}`.

In [12]:
def cube_state_to_faces(state):
    """Convert a state list to a dictionary of labeled faces."""
    n = int(np.sqrt(len(state) / 6))  # cube_n/n/n
    n2 = n ** 2
    labels = f"d{n-1},f0,r0,f{n-1},r{n-1},d0".split(',')
    faces = {}
    for i, l in enumerate(labels):
        face = state[n2 * i : n2 * (i + 1)]
        faces[l] = np.asarray(face).reshape(n, n).tolist()
    return faces


for ptype in ('cube_2/2/2', 'cube_3/3/3'):
    sstate = puzzles.query(f"puzzle_type == '{ptype}'").iloc[0, 1]
    print(ptype)
    pprint(cube_state_to_faces(sstate))
    print()

cube_2/2/2
{'d0': [['F', 'F'], ['F', 'F']],
 'd1': [['A', 'A'], ['A', 'A']],
 'f0': [['B', 'B'], ['B', 'B']],
 'f1': [['D', 'D'], ['D', 'D']],
 'r0': [['C', 'C'], ['C', 'C']],
 'r1': [['E', 'E'], ['E', 'E']]}

cube_3/3/3
{'d0': [['F', 'F', 'F'], ['F', 'F', 'F'], ['F', 'F', 'F']],
 'd2': [['A', 'A', 'A'], ['A', 'A', 'A'], ['A', 'A', 'A']],
 'f0': [['B', 'B', 'B'], ['B', 'B', 'B'], ['B', 'B', 'B']],
 'f2': [['D', 'D', 'D'], ['D', 'D', 'D'], ['D', 'D', 'D']],
 'r0': [['C', 'C', 'C'], ['C', 'C', 'C'], ['C', 'C', 'C']],
 'r2': [['E', 'E', 'E'], ['E', 'E', 'E'], ['E', 'E', 'E']]}



There is a move for each layer, which corresponds to a quarter-twist for that layer.

In [8]:
print("cube_2/2/2")
for m, p in puzzle_info.loc['cube_2/2/2', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

print()

print("cube_3/3/3")
for m, p in puzzle_info.loc['cube_3/3/3', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

cube_2/2/2
f0: (23)(2 19 21 8)(3 17 20 10)(4 6 7 5)
f1: (0 18 23 9)(1 16 22 11)(12 13 15 14)
r0: (1 5 21 14)(3 7 23 12)(8 10 11 9)
r1: (23)(0 4 20 15)(2 6 22 13)(16 17 19 18)
d0: (6 18 14 10)(7 19 15 11)(20 22 23 21)
d1: (23)(0 1 3 2)(4 16 12 8)(5 17 13 9)

cube_3/3/3
f0: (53)(6 44 47 18)(7 41 46 21)(8 38 45 24)(9 15 17 11)(10 12 16 14)
f1: (53)(3 43 50 19)(4 40 49 22)(5 37 48 25)
f2: (0 42 53 20)(1 39 52 23)(2 36 51 26)(27 29 35 33)(28 32 34 30)
r0: (2 11 47 33)(5 14 50 30)(8 17 53 27)(18 24 26 20)(19 21 25 23)
r1: (53)(1 10 46 34)(4 13 49 31)(7 16 52 28)
r2: (53)(0 9 45 35)(3 12 48 32)(6 15 51 29)(36 38 44 42)(37 41 43 39)
d0: (15 42 33 24)(16 43 34 25)(17 44 35 26)(45 51 53 47)(46 48 52 50)
d1: (53)(12 39 30 21)(13 40 31 22)(14 41 32 23)
d2: (53)(0 2 8 6)(1 5 7 3)(9 36 27 18)(10 37 28 19)(11 38 29 20)


As a hint to optimizing solutions, note that these moves satisfy certain relations like:
- `m ** 3 == m ** -1` for any move `m`, and,
- `f0 * f1 * r0 * (f1 ** -1) * (f0 ** -1) == d0`

# Wreath Puzzles #

A `wreath` puzzle is two rings joined at two points, roughly like this:

```
   2   3      8   9
1         4          10
     l          r
0         5          11
   7   6     13  12
```

There will usually be points between the common points, however.

A `wreath_X/Y` puzzle has `X` points in the left ring and `Y` points in the right ring.

In [9]:
# A wreath with six in the left and six in the right
# You can see the rings join at points 0 and 2.
print("wreath_6/6")
for m, p in puzzle_info.loc['wreath_6/6', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

wreath_6/6
l: (9)(0 1 2 3 4 5)
r: (0 6 7 2 8 9)


# Globe Puzzles #
A `globe` puzzle is a sphere with cuts along lines of latitude and longitude. If you poke a hole at the North and South poles, cut along the meridian, and spread it out, it will look like a grid:

```
  | f0  f1  f2  f3  f4  f5  f6  f7
--+-------------------------------
r0| 0   1   2   3   4   5   6   7
r1| 8   9   10  11  12  13  14  15
r2| 16  17  18  19  20  21  22  23
r3| 24  25  26  27  28  29  30  31
```

More precisely, a `globe_M/N` is a sphere with `M` lateral cuts (through latitude) and N radial cuts (through longitude). This gives a `(M + 1) x (2 N)` grid of positions. You could format a color state into a grid like this with something like `np.asarray(state).reshape(m+1, 2*n)`. The above is a `globe_3/4` puzzle.

In [10]:
# Get a solution_state
ss_globe34 = puzzles.query("puzzle_type == 'globe_3/4'").iloc[0, 1]

# Reshape into a grid
np.asarray(ss_globe34).reshape(3+1, 2*4)

array([['A', 'A', 'C', 'C', 'E', 'E', 'G', 'G'],
       ['A', 'A', 'C', 'C', 'E', 'E', 'G', 'G'],
       ['B', 'B', 'D', 'D', 'F', 'F', 'H', 'H'],
       ['B', 'B', 'D', 'D', 'F', 'F', 'H', 'H']], dtype='<U1')

There is a move for each lateral layer (the `r`s) and a move for each half (the `f`s). The `r` moves shift a layer by one position laterally, while the `f` moves shift one half of the globe by a half-twist.

In [11]:
print("globe_3/4")
for m, p in puzzle_info.loc['globe_3/4', 'allowed_moves'].items():
    print(f"{m}: {Permutation(p)}")

globe_3/4
r0: (31)(0 1 2 3 4 5 6 7)
r1: (31)(8 9 10 11 12 13 14 15)
r2: (31)(16 17 18 19 20 21 22 23)
r3: (24 25 26 27 28 29 30 31)
f0: (31)(0 27)(1 26)(2 25)(3 24)(8 19)(9 18)(10 17)(11 16)
f1: (31)(1 28)(2 27)(3 26)(4 25)(9 20)(10 19)(11 18)(12 17)
f2: (31)(2 29)(3 28)(4 27)(5 26)(10 21)(11 20)(12 19)(13 18)
f3: (31)(3 30)(4 29)(5 28)(6 27)(11 22)(12 21)(13 20)(14 19)
f4: (4 31)(5 30)(6 29)(7 28)(12 23)(13 22)(14 21)(15 20)
f5: (0 29)(5 24)(6 31)(7 30)(8 21)(13 16)(14 23)(15 22)
f6: (0 31)(1 30)(6 25)(7 24)(8 23)(9 22)(14 17)(15 16)
f7: (0 25)(1 24)(2 31)(7 26)(8 17)(9 16)(10 23)(15 18)


# Submissions #

The submission format requires moves in a sequence to be delimited by a period `.`. Indicate the inverse of a named move with a preceeding `-`. Moves are applied from left to right.

The `sample_submission.csv` contains a baseline solution for each puzzle.

In [12]:
sample_submission

,moves
id,
0,r1.-f1
1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...
393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....


Let's check that the given sequence for the first puzzle is actually a solution.

In [13]:
def apply_sequence(sequence, moves, state):
    """Apply a sequence of moves in array form to a color state."""
    state = np.asarray(state)
    for m in sequence.split('.'):
        state = state[moves[m]]
    return state


# Convert allowed_moves to dict and add inverse moves
all_moves = puzzle_info.loc[:, 'allowed_moves'].to_dict()
for ptype, moves in all_moves.copy().items():
    for m, arr in moves.copy().items():
        all_moves[ptype][f"-{m}"] = np.argsort(arr).tolist()


# Get info for the first puzzle
solution_state = puzzles.iloc[0, 1]
initial_state = puzzles.iloc[0, 2]
baseline_solution = sample_submission.loc[0, 'moves']

state = apply_sequence(baseline_solution, all_moves['cube_2/2/2'], initial_state)
np.array_equal(state, solution_state)

True

In this case, the resulting state is exactly equal to the required solution state. For puzzles with wildcards, there can be an allowed number of differences. See the competitions [Evaluation](https://www.kaggle.com/competitions/santa-2023/overview/evaluation) page for more info.

# Good luck! #

We hope this introduction was informative! Here are some places to follow up for more information:
- [The Complexity Dynamics of Magic Cubes and Twisty Puzzles](https://resplendence.org/dhushara.com/cubes/cubes.pdf)
- [Twisty Puzzle Museum](https://twistypuzzles.com/app/museum/museum_search.php)
- [Analyzing Rubik's Cube with GAP](https://www.gap-system.org/Doc/Examples/rubik.html) - You could try reproducing this with [SAGE](https://www.sagemath.org/), if you prefer.